In [102]:
import re
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from collections import Counter

In [103]:
# Carga de datos
import pandas as pd

df_repuestos = pd.read_excel(r"C:\Users\danie\OneDrive\IA\Proyecto_Final_IA\data\Lineas de Mantenimiento - repuestos.xlsx")
df_mant = pd.read_excel(r"C:\Users\danie\OneDrive\IA\Proyecto_Final_IA\data\INFORME DE ESTADO DE COMUNICACIONES NO COMUNICACIONES E INFORMATICA ( Br Bl I).xlsx",header=3)

In [104]:
# Mostrar los datos
df_repuestos.head()

,EFECTO (NNE),TIPO DE FALLA,RESEÑA DE LA FALLA,REPUESTOS MAS PROBABLES,REPUESTOS CASI PROBABLES,REPUESTOS POCO PROBABLES
0,MOTOROLA 5050,rx,El equipo transmite pero no recibe,modulo,transistor pnp,NaN
1,MOTOROLA 5050,rx,El equipo transmite pero no recibe,PTT,NaN,NaN
2,MOTOROLA 5050,rx,El equipo transmite pero no recibe,antena,NaN,NaN
3,MOTOROLA 5050,no enciende,El equipo no enciende,placa de alimentacion,bateria,NaN
4,MOTOROLA 5050,no enciende,El equipo no enciende,pin de carga,NaN,NaN


**Tratamiento de los datos**
a) Borrar los los nombres de las columnas invalidas
B) Reemplazar los NaN por 0 - menos la ultima columna


In [105]:
# reemplazar las columnas no validas por la última columna válida anterior
new_cols = []
for i, col in enumerate(df_mant.columns):
    if "Unnamed" in col:
                new_cols.append(new_cols[-1])
    else:
        new_cols.append(col)

df_mant.columns = new_cols

# Eliminos los NaN todas menos la última
df_mant.iloc[:, :-1] = df_mant.iloc[:, :-1].fillna(0)

## MUESTRA DEL DF

In [106]:
df_mant.head()

,N° Ord,NNE,INE,BAL TANDIL,BAL TANDIL,BAL TANDIL,ESC COM BL 1,ESC COM BL 1,ESC COM BL 1,CDO BR BL I,...,JEF AGR AA EJ 601 - EC,JEF AGR AA EJ 601 - EC,JEF AGR AA EJ 601 - EC,G MANT SIS AA 601,G MANT SIS AA 601,G MANT SIS AA 601,TOTAL GENERAL,TOTAL GENERAL,TOTAL GENERAL,INDIQUE FALLAS
0,0,0,0,Cargo,E/S,F/S,Cargo,E/S,F/S,Cargo,...,Cargo,E/S,F/S,Cargo,E/S,F/S,Cargo,E/S,F/S,PRINCIPALES
1,1,3895002526898,EQ DE TEN Y REPL RL 31 P/CABLE TEF CAMP,0,0,0,7,7,0,0,...,0,0,0,0,0,0,8,8,0,NaN
2,2,3895003563937,EJE DE CARRETE DE TEN Y REPL RL 27 D P/CABLE T...,0,0,0,7,7,0,0,...,0,0,0,0,0,0,7,7,0,NaN
3,3,3895005377953,EQ DE TEN Y REPL RL 26 P/CABLE TEF CAMP,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,1,UM (CUENTA MUSEO ) (ESC COM BL 1)
4,4,5805005211320,TELEFONO DE CAMPAÑA PORTATIL TA 1/PT,0,0,0,11,0,11,0,...,0,0,0,0,0,0,11,0,11,UM (CUENTA MUSEO ) (ESC COM BL 1)


## IDENTIFICAR LAS UNIDADES APOYADAS

In [107]:
# Columnas a excluir
excluir = ['N° Ord', 'NNE', 'INE', 'TOTAL GENERAL', 'INDIQUE FALLAS']

# Columnas únicas sin las excluidas
cols_filtradas = df_mant.columns.drop_duplicates().drop(excluir)

# Mostrar en columnas verticales
#Unidades = df_mant[cols_filtradas].nunique()
#print(Unidades)
print("Elementos Militares:")
for col in cols_filtradas:
    print(col)  

Elementos Militares:
BAL TANDIL
ESC COM BL 1
CDO BR BL I
ESC ICIA BL 1
RC TAN 10
GA BL 1
RC TAN 2
ESC ING BL 1
RI MEC 7
ESC EXPL C BL 1
RC TAN 8
GAA MIX 602
GAA 601
JEF AGR AA EJ 601 - EC
G MANT SIS AA 601


### TRATAMIENTO DEL DF PARA SER PROCESADO

In [108]:
# Usar la primera fila como encabezado y eliminar los nombres originales
df_fallas = df_mant[['NNE', 'INE', 'TOTAL GENERAL', 'INDIQUE FALLAS']]
df_fallas.columns = df_fallas.iloc[0] # Elimino la fila 0
df_fallas = df_fallas.iloc[:-2] # Elimino las dos últimas filas
df_fallas = df_fallas[1:]   # Elimino la fila 1

# Cargo los nombres de las columnas
df_fallas.columns.values[0] = "NNE"
df_fallas.columns.values[1] = "INE"
df_fallas.columns.values[5] = "PRINCIPALES FALLAS"
filas=df_fallas.shape[0]
#df_fallas.head(40)

In [109]:
# Error de carga de datos

for idx, row in df_fallas.iterrows():
    if (row["Cargo"] - row["E/S"]) != row["F/S"] :
       print (f"Error en fila {idx}: {row['Cargo']} - {row['E/S']} = {(row['Cargo'] - row['E/S'])} != {row['F/S']}")


Error en fila 5: 37 - 20 = 17 != 0
Error en fila 23: 20 - 9 = 11 != 6
Error en fila 38: 1 - 0 = 1 != 0
Error en fila 55: 19 - 17 = 2 != 12
Error en fila 57: 14322 - 14322 = 0 != 1
Error en fila 115: 16 - 12 = 4 != 2
Error en fila 155: 35 - 22 = 13 != 14


## NORMALIZACIÓN DE LAS FALLAS

In [110]:
# Crear un nuevo DataFrame con columnas NNE y PRINCIPALES FALLAS
fallas_resumido = df_fallas[["INE", "PRINCIPALES FALLAS"]].copy()


In [111]:
#fallas_resumido.head(40)

### Eliminar las fallas que no admiten reparación o efectos que no presentan fallas

In [112]:
# Eliminar las filas con valores NaN en la columna "PRINCIPALES FALLAS"
fallas_resumido = fallas_resumido.dropna(subset=["PRINCIPALES FALLAS"])


In [113]:

# 1) Funciones de normalización


def to_upper(s: pd.Series) -> pd.Series:
    return s.str.upper()

def strip_spaces(s: pd.Series) -> pd.Series:
    return s.str.strip()

def normalize_spaces(s: pd.Series) -> pd.Series:
    s = s.str.replace(r"\s+", " ", regex=True)
    s = s.str.replace(r"\(\s+", "(", regex=True)
    s = s.str.replace(r"\s+\)", ")", regex=True)
    s = s.str.replace(r"\s*,\s*", ", ", regex=True)
    s = s.str.replace(r"\s*-\s*", " - ", regex=True)
    s = s.str.replace(r"\s*:\s*", ": ", regex=True)
    return s

# Regex para split (no corta fechas ni F/S o E/S)
regex_split = r'(?<!\d)\s*-\s*(?!\d)(?!\s*\d)|(?<!\d)(?<!F)(?<!E)(?<!S)/(?!\d)|\s+Y\s+|\s*,\s*'


def split_and_clean(s: pd.Series) -> pd.Series:
    return s.apply(
        lambda x: [p.strip() for p in re.split(regex_split, x) if p.strip()] if pd.notna(x) else []
    )


# 2) Pipeline


normalizer = Pipeline(steps=[
    ("to_upper", FunctionTransformer(to_upper)),
    ("strip_spaces", FunctionTransformer(strip_spaces)),
    ("normalize_spaces", FunctionTransformer(normalize_spaces)),
    ("split", FunctionTransformer(split_and_clean, validate=False)),
])


# 3) Uso sobre tu DataFrame

fallas_resumido["PRINCIPALES FALLAS"] = normalizer.fit_transform(fallas_resumido["PRINCIPALES FALLAS"])
#fallas_resumido["INE"] = normalizer.fit_transform(fallas_resumido["INE"])



In [114]:
# despliegue la columna principales fallas en varias filas pero manteniendo la columna INE por cada parte de la lista de las filas 
fallas_resumido = fallas_resumido.explode("PRINCIPALES FALLAS")

###  NUEVO EXCEL CON EL RESUMEN DE LAS FALLAS POR INE

In [115]:
# Guardar el DataFrame en un nuevo archivo Excel
fallas_resumido.to_excel("resumen_fallas_ppl.xlsx", index=False)

# Contar frecuencia de fallas

conteo_fallas = fallas_resumido.value_counts()

print("Fallas declaradas:")
n=1
for falla, cantidad in conteo_fallas.items():
    print(f" {n}) {falla}: {cantidad}")
    n += 1

# Guardar el conteo en un nuevo archivo Excel
conteo_fallas_df = conteo_fallas.reset_index(name='Cantidad')
conteo_fallas_df.to_excel("fallas.xlsx", index=False)

### DICCIONARIO DE FALLAS - CATEGORIZACION

In [119]:

# Lista de códigos a excluir
excluir = ["COM", "BL", "RC", "RI", "GA", "GAA", "ING", "TAN", "ESC", "OM", "EXPL", "MEC", "602","AA", "B", "EJ", "B MANT", "601", "137", "NRO"
           "DOS", "UNO", "TRES", "CUATRO", "CINCO", "SEIS", "SIETE", "OCHO", "NUEVE", "DIEZ", "F","1","2","3","4","5","6","7","8","9","10","S","NRO"]

frases = []
for texto in fallas_resumido["PRINCIPALES FALLAS"].dropna():
    limpio = re.sub(r"[^A-Z0-9ÁÉÍÓÚÑ ]", " ", texto.upper())
    tokens = limpio.split()
    for n in [2, 3]:
        for i in range(len(tokens)-n+1):
            frase = " ".join(tokens[i:i+n])
            # ❌ filtramos si contiene alguna palabra prohibida
            if any(bad in frase.split() for bad in excluir):
                continue
            # ❌ filtramos si es solo números
            if frase.replace(" ", "").isdigit():
                continue
            # ✅ si pasa, lo guardamos
            frases.append(frase)
        
# Contamos frecuencia
frases_comunes = Counter(frases).most_common(50)

# Mostramos las frases más comunes
for frase, freq in frases_comunes:
    print(f"{frase}: {freq}")   


UM CUENTA: 11
CUENTA MUSEO: 11
UM CUENTA MUSEO: 11
NO SINTONIZA: 11
BATERÍA BAJO: 9
BAJO RENDIMIENTO: 9
BATERÍA BAJO RENDIMIENTO: 9
MICROTELEFONO H189: 6
DOS 02: 6
NO ENCIENDE: 6
NO TX: 5
NO RETIENEN: 5
RETIENEN CARGA: 5
NO RETIENEN CARGA: 5
NO RETIENE: 5
RETIENE CARGA: 5
NO RETIENE CARGA: 5
H189 INOPERABLE: 5
MICROTELEFONO H189 INOPERABLE: 5
CABLE DE: 5
REMITIDO AL: 5
FALTA OPTIMIZAR: 5
OPTIMIZAR PARA: 5
PARA USO: 5
USO CORRECTO: 5
CORRECTO WIN10: 5
FALTA OPTIMIZAR PARA: 5
OPTIMIZAR PARA USO: 5
PARA USO CORRECTO: 5
USO CORRECTO WIN10: 5
DE ANTENA: 4
FUERA DE: 4
REMITIDO PARA: 4
TX NO: 4
NO RX: 4
NO TX NO: 4
TX NO RX: 4
ENTREGADO A: 4
A U3449: 4
U3449 L3: 4
L3 SEGÚN: 4
SEGÚN SRE137: 4
SRE137 NOTA: 4
NOTA RECIBO: 4
RECIBO 24: 4
ENTREGADO A U3449: 4
A U3449 L3: 4
U3449 L3 SEGÚN: 4
L3 SEGÚN SRE137: 4
SEGÚN SRE137 NOTA: 4


In [ ]:
def categorizar_falla(texto: str, diccionario: dict) -> list:
    if pd.isna(texto):
        return []
    texto = texto.upper()
    categorias = []
    # buscamos frases largas primero
    for clave, categoria in sorted(diccionario.items(), key=lambda x: -len(x[0])):
        if clave in texto:
            categorias.append(categoria)
    return list(set(categorias))  # eliminamos duplicados
